In [3]:
# basics
import numpy as np
import pandas as pd
import datetime
import re
import os,glob
import pickle
import textwrap

# statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf


# plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [15,12]

# own data wrappers
from imp import reload
import measureclass as mc; reload(mc);
import coronadataclass as cdc; reload(cdc);
import crossvalidationclass as cvc; reload(cvc);

np.seterr(divide = 'ignore');


In [4]:
cccsl     = mc.COVID19_measures(datasource = 'CCCSL',     download_data = True)
oxford    = mc.COVID19_measures(datasource = 'OXFORD',    download_data = True)
whophsm   = mc.COVID19_measures(datasource = 'WHOPHSM',   download_data = True)
acaps     = mc.COVID19_measures(datasource = 'ACAPS',     download_data = True)
coronanet = mc.COVID19_measures(datasource = 'CORONANET', download_data = True)
hitcovid  = mc.COVID19_measures(datasource = 'HITCOVID',  download_data = True)

In [ ]:
print('           {:8s} {:9s}'.format('Measures','Countries'))
print('CCCSL:     {:8d} {:9d}'.format(cccsl.MeasureList(mincount = 5).shape[0],     len(cccsl.countrylist)))
print('OXFORD:    {:8d} {:9d}'.format(oxford.MeasureList(mincount = 5).shape[0],    len(oxford.countrylist)))
print('WHOPHSM:   {:8d} {:9d}'.format(whophsm.MeasureList(mincount = 5).shape[0],   len(whophsm.countrylist)))
print('ACAPS:     {:8d} {:9d}'.format(acaps.MeasureList(mincount = 5).shape[0],     len(acaps.countrylist)))
print('CORONANET: {:8d} {:9d}'.format(coronanet.MeasureList(mincount = 5).shape[0], len(coronanet.countrylist)))
print('HITCOVID:  {:8d} {:9d}'.format(hitcovid.MeasureList(mincount = 5).shape[0],  len(hitcovid.countrylist)))


In [ ]:
print('           {:8s} {:9s}'.format('Measures','Countries'))
print('CCCSL:     {:8d} {:9d}'.format(cccsl.MeasureList().shape[0],     len(cccsl.countrylist)))
print('OXFORD:    {:8d} {:9d}'.format(oxford.MeasureList().shape[0],    len(oxford.countrylist)))
print('WHOPHSM:   {:8d} {:9d}'.format(whophsm.MeasureList().shape[0],   len(whophsm.countrylist)))
print('ACAPS:     {:8d} {:9d}'.format(acaps.MeasureList().shape[0],     len(acaps.countrylist)))
print('CORONANET: {:8d} {:9d}'.format(coronanet.MeasureList().shape[0], len(coronanet.countrylist)))
print('HITCOVID:  {:8d} {:9d}'.format(hitcovid.MeasureList().shape[0],  len(hitcovid.countrylist)))


In [ ]:
# generate a full table for a database

def AddDF(df1,df2):
    if not df1 is None:
        if df2 is None:
            return df1
        else:
            return pd.concat([df1,df2], ignore_index = True, sort = False)
    else:
        return df2
    

finaltable = None
for country in cccsl.countrylist:
    country_implementation_table = cccsl.ImplementationTable(country = country, mincount = 5).reset_index().rename(columns = {'index':'Date'})
    country_implementation_table['country'] = country
    finaltable = AddDF(finaltable, country_implementation_table)
finaltable.fillna(0, inplace = True)

finaltable

In [ ]:
external_cccsl = pd.read_csv('COVID19_data_PAPER_VERSION.csv', sep = ';')
external_whophsm = pd.read_csv('COVID19_data_PAPER_VERSION_WHOPHSM.csv', sep = ';')
external_coronanet = pd.read_csv('COVID19_data_PAPER_VERSION_CORONANET.csv', sep = ';')
external_oxford = pd.read_csv('COVID19_data_PAPER_VERSION_OXFORD.csv', sep = ';')

In [ ]:
oxford_ext_cols = list(external_oxford.columns)
for measureID in oxford.MeasureList(mincount=5).index:
    if measureID in oxford_ext_cols:
        print('found:      {}'.format(measureID))
        oxford_ext_cols.remove(measureID)
    else:
        print('not found:  {}'.format(measureID))
print('remaining: ', oxford_ext_cols)
print('difference:', set(oxford.MeasureList().index).difference(set(oxford.MeasureList(mincount=5).index)))

In [ ]:
cccsl_ext_cols = list(external_cccsl.columns)
for measureID,measuredata in cccsl.MeasureList(mincount=5).iterrows():
    mID = measuredata['Measure_L1'] + ' -- ' + measuredata['Measure_L2']
    if mID in cccsl_ext_cols:
        print('found:      {}'.format(mID))
        cccsl_ext_cols.remove(mID)
    else:
        print('not found:  {}'.format(mID))
print('remaining: ', cccsl_ext_cols)
print('difference:', set(cccsl.MeasureList().index).difference(set(cccsl.MeasureList(mincount=5).index)))

In [ ]:
whophsm_ext_cols = list(external_whophsm.columns)
for measureID in whophsm.MeasureList(mincount = 5).index:
    if measureID in whophsm_ext_cols:
        print('found:      {}'.format(measureID))
        whophsm_ext_cols.remove(measureID)
    else:
        print('not found:  {}'.format(measureID))
print('remaining: ', whophsm_ext_cols)
print('difference:', set(whophsm.MeasureList().index).difference(set(whophsm.MeasureList(mincount=5).index)))

In [ ]:
coronanet_ext_cols = list(external_coronanet.columns)
for measureID in coronanet.MeasureList(mincount = 5).index:
    if measureID in coronanet_ext_cols:
        print('found:      {}'.format(measureID))
        coronanet_ext_cols.remove(measureID)
    else:
        print('not found:  {}'.format(measureID))
print('remaining: ', coronanet_ext_cols)
print('difference:', set(coronanet.MeasureList().index).difference(set(coronanet.MeasureList(mincount=5).index)))